In [12]:
from torchvision import datasets, transforms, utils
from torch.utils import data
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
import numpy as np
import torch
import math
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

In [13]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = datasets.FashionMNIST(root = './.data/', train = True, download = True, transform = transform)
testset = datasets.FashionMNIST(root = './.data/', train = False, download = False, transform = transform)


In [14]:
batch_size = 16
device = 'cuda' if cuda.is_available() else 'cpu'
train_loader = data.DataLoader(dataset = trainset, batch_size = batch_size)
test_loader = data.DataLoader(dataset = testset, batch_size = batch_size)

In [18]:
#module making
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 240)
        self.l3 = nn.Linear(240, 360)
        self.l4 = nn.Linear(360, 100)
        self.l5 = nn.Linear(100, 10)
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)
    
model = net()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [19]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        output = model(data)
        loss = criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if batch_idx % 10 == 0:
        print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [23]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        
        test_loss += criterion(output, target).item()
        
        pred = output.data.max(1, keepdim = True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [ ]:
for epoch in range(1, 15):
    train(epoch)
    test()

Test set: Average loss: 0.0300, Accuracy: 8263/10000 (83%)
Test set: Average loss: 0.0282, Accuracy: 8404/10000 (84%)
Test set: Average loss: 0.0269, Accuracy: 8476/10000 (85%)
Test set: Average loss: 0.0264, Accuracy: 8521/10000 (85%)
Test set: Average loss: 0.0243, Accuracy: 8617/10000 (86%)
Test set: Average loss: 0.0230, Accuracy: 8678/10000 (87%)
Test set: Average loss: 0.0236, Accuracy: 8658/10000 (87%)
Test set: Average loss: 0.0230, Accuracy: 8715/10000 (87%)
Test set: Average loss: 0.0225, Accuracy: 8754/10000 (88%)
Test set: Average loss: 0.0231, Accuracy: 8737/10000 (87%)
Test set: Average loss: 0.0237, Accuracy: 8722/10000 (87%)
Test set: Average loss: 0.0241, Accuracy: 8720/10000 (87%)
Test set: Average loss: 0.0232, Accuracy: 8749/10000 (87%)


In [ ]:
# For when i dont remember detail -> https://bigdatadiary0819.tistory.com/54